In [ ]:
import numpy as np
import numpy.linalg as lg 
from scipy.sparse.linalg import cg
from scipy.linalg import blas
from time import time
from tabulate import tabulate
from  scipy.io import mmread
A1=np.array(mmread('/content/bcsstk14.mtx.gz').todense())
# A1.shape=(1806, 1806) real symmetric positive definite
A2=np.array(mmread('/content/bcsstm19.mtx.gz').todense())
# A2.shape=(817,817) real symmetric positive definite
A3=np.array(mmread('/content/bcsstm20.mtx.gz').todense())
# A3.shape=(485,485) real symmetric positive definite
A4=np.array(mmread('/content/bcsstm24.mtx.gz').todense())
# A4.shape=(3562,3562) real symmetric positive definite
B1=np.array(mmread('/content/bcsstm21.mtx.gz').todense())
B2=np.array(mmread('/content/bcsstk16.mtx.gz').todense())
B3=np.array(mmread('/content/bcsstk17.mtx.gz').todense())
B4=np.array(mmread('/content/bcsstk18.mtx.gz').todense())
C1=np.array(mmread('/content/bcsstm22.mtx.gz').todense())
C2=np.array(mmread('/content/bcsstk27.mtx.gz').todense())
C3=np.array(mmread('/content/bcsstm26.mtx.gz').todense())

## The Conjugate Gradient Algorithm

In [ ]:
def cg_iter(A, b,x0,maxiter,tol):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters += 1
    return cg(A, b, callback=callback,tol = tol,maxiter=maxiter),num_iters

def The_Conjugate_Gradient_Algorithm(A,b,x,itermax,tol): 
    r    = p = b- blas.dgemv(1.0, A, x)
    k    = 0   
    while blas.dnrm2(r) > tol and k<itermax:    
        alpha =  blas.dnrm2(r)**2/( blas.ddot(blas.dgemv(1.0, A, p), p))      
        x     = x + alpha*p
        rpr   = r.copy()        
        r     = rpr - blas.dgemv(alpha, A, p)
        beta  = (blas.dnrm2(r))**2/blas.dnrm2(rpr)**2
        p     = r + beta*p
        k     = k + 1   
    return x, k


# **Cluster 1:**


In [ ]:
matricies=[B1,A2,A3,B2,C1,C2]
tol=1e-5
itermax = 1000

In [ ]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = cg_iter(matricies[i],b,x,itermax,tol)[1]  
  y=cg(matricies[i],b,x,tol,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1=time()
  x,iterm  = The_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  t2=time()
  tmp=(t2-t1)*1000              # time in ms
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(res)
  l.append(tmp)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','res','time(ms)','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 


+--------------+--------------------+----------------+-------------+------------+-------------+------------------+-------------+-------------+
| matricies    |   condition number |   Our_nbr_iter |         res |   time(ms) |     Our_err |   scipy_nbr_iter |      sc_res |   scipy_err |
+==============+====================+================+=============+============+=============+==================+=============+=============+
| (3600, 3600) |       23.7154      |              3 | 1.52728e-12 |   955.21   | 1.40091e-08 |                3 | 2.1765e-12  | 1.64624e-08 |
+--------------+--------------------+----------------+-------------+------------+-------------+------------------+-------------+-------------+
| (817, 817)   |   233734           |            485 | 7.97427e-06 |  3904.03   | 1.86289e-08 |              393 | 0.00022379  | 2.69577e-07 |
+--------------+--------------------+----------------+-------------+------------+-------------+------------------+-------------+-------------+

# **Cluster 2:**

In [ ]:
matricies=[A1,A4,C3]
tol=1e-5
itermax = 30000

In [ ]:
my_data = []
for i in range(len(matricies)):
  l = []
  n = len(matricies[i])
  b=np.ones(n)
  x = np.zeros(n)
  sol  = lg.solve(matricies[i], b)
  itersc = cg_iter(matricies[i],b,x,itermax,tol)[1]  
  y=cg(matricies[i],b,x,tol,itermax)[0]
  sc_err=np.linalg.norm(y-sol)
  sc_res=lg.norm(np.dot(matricies[i],y)-b)
  t1=time()
  x,iterm  = The_Conjugate_Gradient_Algorithm(matricies[i],b,x,itermax,tol)
  t2=time()
  tmp=(t2-t1)*1000              # time in ms
  err=lg.norm(x-sol)
  res=lg.norm(np.dot(matricies[i],x)-b)
  l.append(matricies[i].shape)
  l.append(lg.cond(matricies[i]))
  l.append(iterm)
  l.append(res)
  l.append(tmp)
  l.append(err)
  l.append(itersc)
  l.append(sc_res)
  l.append(sc_err)
  my_data.append(l)
# create header
head = ['matricies','condition number','Our_nbr_iter','res','time(ms)','Our_err','scipy_nbr_iter','sc_res','scipy_err']
  
# display table
print(tabulate(my_data, headers=head, tablefmt="grid")) 


+--------------+--------------------+----------------+-------------+------------------+-------------+------------------+-------------+---------------+
| matricies    |   condition number |   Our_nbr_iter |         res |         time(ms) |     Our_err |   scipy_nbr_iter |      sc_res |     scipy_err |
+==============+====================+================+=============+==================+=============+==================+=============+===============+
| (1806, 1806) |        1.19232e+10 |          14170 | 9.52212e-06 | 919354           | 8.26869e-11 |            12822 | 0.000386238 |   2.48844e-09 |
+--------------+--------------------+----------------+-------------+------------------+-------------+------------------+-------------+---------------+
| (3562, 3562) |        1.80764e+13 |          23589 | 9.95552e-06 |      5.95875e+06 | 5.43079     |            18229 | 0.000538547 | 527.254       |
+--------------+--------------------+----------------+-------------+------------------+-------